<a href="https://colab.research.google.com/github/jeregalon/deteccion_yolo_segment/blob/main/Detector_de_pantallas_con_segmentaci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Entrenamiento del modelo de visión artificial YOLOv11 para detectar objetos con segmentación.

En este caso entreno el modelo para detectar pantallas de pesas digitales y otros instrumentos de medición, utilizando un dataset de elaboración propia hecho con LabelStudio.

In [ ]:
# Importar el dataset desde Google Drive

from google.colab import drive
drive.mount('/content/gdrive')

!cp /content/gdrive/MyDrive/data_seg.zip /content
!cp /content/gdrive/MyDrive/yolo11n-seg.pt /content

In [ ]:
!unzip -q /content/data_seg.zip -d /content/custom_data

In [ ]:
# Dividiendo el dataset en datos de entrenamiento y datos de validación

# El archivo train_val_split.py se encuentra en https://github.com/jeregalon/deteccion_yolo_segment/tree/main

!python train_val_split.py --datapath="/content/custom_data/data" --train_pct=0.9

In [ ]:
!pip install ultralytics

In [ ]:
# Python function to automatically create data.yaml config file
# 1. Reads "classes.txt" file to get list of class names
# 2. Creates data dictionary with correct paths to folders, number of classes, and names of classes
# 3. Writes data in YAML format to data.yaml

import yaml
import os

def create_data_yaml(path_to_classes_txt, path_to_data_yaml):

  # Read class.txt to get class names
  if not os.path.exists(path_to_classes_txt):
    print(f'classes.txt file not found! Please create a classes.txt labelmap and move it to {path_to_classes_txt}')
    return
  with open(path_to_classes_txt, 'r') as f:
    classes = []
    for line in f.readlines():
      if len(line.strip()) == 0: continue
      classes.append(line.strip())
  number_of_classes = len(classes)

  # Create data dictionary
  data = {
      'path': '/content/data',
      'train': 'train/images',
      'val': 'validation/images',
      'nc': number_of_classes,
      'names': classes
  }

  # Write data to YAML file
  with open(path_to_data_yaml, 'w') as f:
    yaml.dump(data, f, sort_keys=False)
  print(f'Created config file at {path_to_data_yaml}')

  return

# Define path to classes.txt and run function
path_to_classes_txt = '/content/custom_data/data/classes.txt'
path_to_data_yaml = '/content/data.yaml'

create_data_yaml(path_to_classes_txt, path_to_data_yaml)

print('\nFile contents:\n')
!cat /content/data.yaml

In [ ]:
# Entrenamiento del modelo

!yolo obb train data=/content/data.yaml model=yolo11n-seg.pt epochs=200 imgsz=640

# Si el código anterior te da error, puedes descargar el modelo desde la página
# de Ultralytics y añadirlo a los archivos. Está disponible en:
# https://docs.ultralytics.com/es/tasks/segment/

# Cuando lo descargues y lo tengas entre tus archivos, ejecuta el siguiente código
# en vez del anterior:

# !yolo segment train data=/content/data.yaml model=/content/yolo11n-seg.pt epochs=200 imgsz=640



In [ ]:
!yolo segment predict model=runs/segment/train/weights/best.pt source=data/validation/images save=True

In [ ]:
import glob
from IPython.display import Image, display
for image_path in glob.glob(f'/content/segment/detect/predict/*.jpg')[:10]:
  display(Image(filename=image_path, height=400))
  print('\n')

In [ ]:
# Create "my_model" folder to store model weights and train results
!mkdir /content/my_model
!cp /content/runs/segment/train/weights/best.pt /content/my_model/my_model.pt
!cp -r /content/runs/segment/train /content/my_model

mkdir: cannot create directory ‘/content/my_model’: File exists


In [ ]:
# Zip into "my_model.zip"
%cd my_model
!zip /content/my_model.zip my_model.pt
!zip -r /content/my_model.zip train
%cd /content

In [ ]:
# Descargar el modelo a la computadora
from google.colab import files

files.download('/content/my_model.zip')